In [8]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from fedlab.contrib.algorithm.basic_client import SGDSerialClientTrainer
from fedlab.contrib.algorithm.basic_server import SyncServerHandler
from fedlab.core.standalone import StandalonePipeline
from fedlab.utils.functional import evaluate

Possible features to implement:

- Transaction History
    - Amt, Timestamps -> Frequency
- Current Balance
- User interaction
- Geolocation Data
- Time patterns
- How often users redeem rewards
- Wallet features used
- Financial Goals

-Some attributes are categorical like user interaction. We might have to do some sort of engagement leveling for that.
- Wallet features can indicate what kind of resources a user might desire:
    - If they like to check their balance more than making transactions, it might be a sign that a user is considering making a purchase but is nervous about consequences regarding it. This could be "scenario 1" and can be encoded as a one hot vector like [1, 0, 0, ..., 0]

### NOTE

- Most features are tentative and may not be implemented. It is unclear as to what kind of data we will have access to at the current moment and whether or not hte collection of this data is feasible.

In [ ]:
# Prototype read data function.
df = pd.read_csv("user-data.csv", delimiter=";")

# Data matrix
D = df.to_numpy()

# Presumably, (# of points, 8 + # of wallet features)
print(D.shape)


# It is likely that we will have to make our own assessments on each dataset.
X, Y = D[:, :-1], D[:, -1]

# Feature Programming

In [3]:
# Transaction history, columns 0 and 1

def spendings(Amt, Timestamps, Balances):
    # I don't know if what we'll be storing are datetime objects so I'm
    # just going to be careful and say no.
    Timestamps = pd.to_datetime(Timestamps)

    # Calculate difference in minutes
    time_diffs = np.diff(Timestamps).astype('timedelta64[m]').astype(int)

    # Normalization.
    norm_time_diffs = 1 / (time_diffs + 1)
    norm_balances = (Balances - Amt) / Balances

    # Combine the scores: lower values for both indicate potentially reckless spending
    return norm_balances * norm_time_diffs

def normalize(v):
    return 2 * ( (v - np.amin(v)) / (np.amax(v) - np.amin(v)) ) - 1

In [ ]:
def attribute_matrix(X):
    # Spendings dictates how much of their balance is spent per transaction
    # within a certain time interval.
    a1 = spendings(X[:, 0], X[:, 1], X[:, 2])

    # Normalized number of queries (user interaction)
    a2 = normalize(v=X[:, 3])

    # TODO: Other features...

    return np.column_stack((a1, a2))

In [ ]:
# # This is code heavily based on Zaki's implementation of a Simple Neural Network.

# def relu(z):
#     """Apply the ReLU (Rectified Linear Unit) function."""
#     return np.maximum(0, z)

# def relu_derivative(z):
#     """Compute the derivative of the ReLU function."""
#     return np.where(z > 0, 1, 0)

# def feed_forward(x, network):
#     """Perform a feedforward pass through the neural network."""
#     activations = [x]
#     input_to_layer = x

#     for layer in network:
#         z = layer['b'] + np.dot(layer['W'].T, input_to_layer)
#         input_to_layer = relu(z)
#         activations.append(input_to_layer)

#     activations[-1] = softmax(activations[-1])
#     return activations

# def initialize_network(input_size, hidden_layer_sizes, output_size, scale):
#     """Initialize a deep multilayer perceptron with random weights and biases."""
#     layer_sizes = [input_size] + hidden_layer_sizes + [output_size]
#     network = []

#     for i in range(len(layer_sizes) - 1):
#         layer = {
#             'b': np.random.rand(layer_sizes[i + 1]) * scale,
#             'W': np.random.rand(layer_sizes[i], layer_sizes[i + 1]) * scale
#         }
#         network.append(layer)

#     return network

# def deep_mlp_training(data, output_size, max_iter, learning_rate, hidden_layer_sizes, scale):
#     """Train a deep multilayer perceptron on the given dataset."""
#     num_samples, num_features = data.shape
#     input_size = num_features - 1  # Last column is assumed to be the label
#     network = initialize_network(input_size, hidden_layer_sizes, output_size, scale)

#     for j in range(max_iter):
#         indices = np.arange(num_samples)
#         np.random.shuffle(indices)

#         for i in indices:
#             x_i = data[i, :-1]
#             y_i = np.zeros(output_size)
#             y_i[int(data[i, -1])] = 1

#             # Forward pass
#             activations = feed_forward(x_i, network)

#             # Backpropagation
#             deltas = [activations[-1] - y_i]
#             for l in range(len(network) - 1, 0, -1):
#                 delta = relu_derivative(np.dot(network[l]['W'], deltas[0]))
#                 deltas.insert(0, delta)

#             # Gradient descent parameter update
#             for l, layer in enumerate(network):
#                 layer['W'] -= learning_rate * np.outer(activations[l], deltas[l])
#                 layer['b'] -= learning_rate * deltas[l]

#     return network


In [ ]:
class DeepMLP(nn.Module):
    def __init__(self, input_size=0, hidden_layer_sizes=0, output_size=0):
        super(DeepMLP, self).__init__()
        layers = []

        # Input layer
        layers.append(nn.Linear(input_size, hidden_layer_sizes[0]))
        layers.append(nn.ReLU())

        # Hidden layers
        for i in range(len(hidden_layer_sizes)-1):
            layers.append(nn.Linear(hidden_layer_sizes[i], hidden_layer_sizes[i+1]))
            layers.append(nn.ReLU())

        # Output layer
        layers.append(nn.Linear(hidden_layer_sizes[-1], output_size))

        # Combine all layers
        self.layers = nn.Sequential(*layers)

    # Feed Forward.
    def forward(self, x):
        return self.layers(x)


In [ ]:
class EvalPipeline(StandalonePipeline):
    def __init__(self, handler, trainer, test_loader, show_data=True):
        super().__init__(handler, trainer)
        self.show_data = show_data
        self.test_loader = test_loader
        self.loss, self.acc = [], []
        self.ax = None
        self.ax2 = None

    def getLoss(self):
        return self.loss
    
    def getAcc(self):
        return self.acc

    def main(self):
        t = 0
        while not self.handler.if_stop:
            self.trainer.local_process(self.handler.downlink_package,
                                       self.handler.sample_clients())
            
            for pack in self.trainer.uplink_package:
                self.handler.load(pack)
            
            loss, acc = evaluate(self.handler.model, 
                                 nn.CrossEntropyLoss(),
                                 self.test_loader)
            if (self.show_data):
                print(f"Round {t}, Loss {round(loss,4)}, Test Acc {round(acc,4)}")

            self.loss.append(loss)
            self.acc.append(acc)
            
            t += 1

    def show(self):
        plt.figure(figsize=(8,4.5))
        self.ax = plt.subplot(1,2,1)
        self.ax.plot(np.arange(len(self.loss)), self.loss)
        self.ax.set_xlabel("Communication Round")
        self.ax.set_ylabel("Loss")
        
        self.ax2 = plt.subplot(1,2,2)
        self.ax2.plot(np.arange(len(self.acc)), self.acc)
        self.ax2.set_xlabel("Communication Round")
        self.ax2.set_ylabel("Accuracy")

In [ ]:
# Just like in the lab, we need to implement client-side training and server side aggregation.
# I simply copy and pasted from my code:

class NetworkOptions():
    def __init__(self, input_size, hidden_layer_sizes, output_size):
        self.input_size = input_size
        self.hidden_layer_sizes = hidden_layer_sizes
        self.output_size = output_size

def run(
        training_data: np.ndarray,
        test_data: np.ndarray,
        input_size: int, 
        hidden_layer_sizes: int,
        output_size: int,
        epochs=1,
        batch_size=1024,
        eta=0.04,
        cuda=True,
        num_rounds=100,
        num_clients=10
    ):
    # TODO: Get data lol

    # Convert numpy arrays to PyTorch tensors
    training_data = torch.tensor(training_data, dtype=torch.float32)
    test_data = torch.tensor(test_data[:, -1], dtype=torch.long) 

    model = DeepMLP(input_size, hidden_layer_sizes, output_size)

    # Create an instance of the trainer for serial training on clients
    trainer = SGDSerialClientTrainer(model=model,
                                    num_clients=num_clients,
                                    cuda=cuda)


    # Once we actually HAVE the data, you can set it up as follows:
    trainer.setup_dataset(training_data)

    # Setup optimizer with the defined epochs, batch size, and learning rate
    trainer.setup_optim(epochs=epochs,
                        batch_size=batch_size,
                        lr=eta)

    handler = SyncServerHandler(model=model, 
                                global_round=num_rounds,
                                sample_ratio=0.1)

    # TODO: I can't believe it, we STILL don't have data.
    test_loader = DataLoader(test_data, batch_size=batch_size)
    standalone_eval = EvalPipeline(handler=handler, trainer=trainer, test_loader=test_loader)
    standalone_eval.main()
    standalone_eval.show()